In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm


# -------------------- Parameters --------------------
CL = 0.99
z = 2.3263478740408408
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2])  # must align with Asset_A, Asset_B, Asset_C

# Load returns
returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)
assets = list(returns.columns)

# -------------------- Base 1d Normal VaR --------------------
cov = returns.cov().values
sigma_p = float(np.sqrt(weights @ cov @ weights.T))
base_var_1d_normal = z * sigma_p * total_value

# -------------------- Liquidity parameters --------------------
# Placeholder values (user can update with actual HalfSpread, TTL_Days, Impact_bps)
values = total_value * weights
liq_params = pd.DataFrame({
    "Asset": assets,
    "Value": values,
    "HalfSpread_bps": [10.0, 15.0, 8.0],  # example half spreads in basis points
    "TTL_Days": [3, 3, 3],                 # example time-to-liquidate
    "Impact_bps": [5.0, 8.0, 4.0],         # example impact cost in bps
})

# -------------------- Liquidity Add-on calculations --------------------
liq_params["AddOn_spread_sqrtTTL"] = (
    (liq_params["HalfSpread_bps"] / 1e4) * liq_params["Value"] * np.sqrt(liq_params["TTL_Days"])
)
liq_params["AddOn_impact"] = (
    (liq_params["Impact_bps"] / 1e4) * liq_params["Value"] * np.sqrt(liq_params["TTL_Days"])
)

liquidity_addon_total = float(liq_params["AddOn_spread_sqrtTTL"].sum() + liq_params["AddOn_impact"].sum())
lvar_1d_total = base_var_1d_normal + liquidity_addon_total

# -------------------- Output --------------------
asset_table = liq_params.set_index("Asset")
metrics_table = pd.DataFrame({
    "Metric": ["Base_VaR_1d_Normal", "Liquidity_AddOn_Total", "LVaR_1d_Total"],
    "Value": [base_var_1d_normal, liquidity_addon_total, lvar_1d_total]
}).set_index("Metric")

# Save
asset_csv = "sample_data/liquidity_addons_assets.csv"
metrics_csv = "sample_data/liquidity_addons_metrics.csv"
asset_table.round(2).to_csv(asset_csv)
metrics_table.round(2).to_csv(metrics_csv)

# Display rounded
print("Liquidity Add-ons (per asset)", asset_table.round(2))
print("Liquidity Metrics", metrics_table.round(2))




Liquidity Add-ons (per asset)                    Value  HalfSpread_bps  TTL_Days  Impact_bps  \
Asset                                                            
Asset_A_Return  500000.0            10.0         3         5.0   
Asset_B_Return  300000.0            15.0         3         8.0   
Asset_C_Return  200000.0             8.0         3         4.0   

                AddOn_spread_sqrtTTL  AddOn_impact  
Asset                                               
Asset_A_Return                866.03        433.01  
Asset_B_Return                779.42        415.69  
Asset_C_Return                277.13        138.56  
Liquidity Metrics                           Value
Metric                         
Base_VaR_1d_Normal     20187.01
Liquidity_AddOn_Total   2909.85
LVaR_1d_Total          23096.86
